In [5]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()
client_id = os.getenv("client_id_event")

# 1. Load venue IDs from CSV
def load_venue_ids(csv_path=r"events_data\venues_data\KC_venues.csv"):
    df = pd.read_csv(csv_path)
    return df["id"].dropna().astype(int).tolist()

# 2. Calculate date range: 8 days out, for 7 days
def get_date_range():
    start_date = datetime.now().date() + timedelta(days=8)
    end_date = start_date + timedelta(days=6)
    return start_date.isoformat(), end_date.isoformat()

# 3. Fetch events for a single venue
def fetch_events_for_venue(venue_id, start_date, end_date):
    url = "https://api.seatgeek.com/2/events"
    params = {
        "venue.id": venue_id,
        "datetime_utc.gte": f"{start_date}T00:00:00",
        "datetime_utc.lte": f"{end_date}T23:59:59",
        "client_id": client_id
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get("events", [])
    else:
        print(f"❌ Error for venue {venue_id}: {response.status_code}")
        return []

# 4. Main loop
def fetch_all_events():
    venue_ids = load_venue_ids()
    start_date, end_date = get_date_range()
    all_events = []

    for vid in venue_ids:
        events = fetch_events_for_venue(vid, start_date, end_date)
        all_events.extend(events)
        print(f"✅ {len(events)} events pulled for venue {vid}")

    return all_events



In [6]:
# 5. Flatten and save
def save_events(events, folder="events_data/KC_events/Raw"):
    if not events:
        print("⚠️ No events to save.")
        return

    # Ensure folder exists
    os.makedirs(folder, exist_ok=True)

    # Generate timestamped filename
    timestamp = datetime.now().strftime('%Y%m%d')
    filename = f"events_kc_{timestamp}.csv"
    filepath = os.path.join(folder, filename)

    # Normalize and save
    df = pd.json_normalize(events)
    df.to_csv(filepath, index=False)
    print(f"✅ Saved {len(df)} events to {filepath}")



# Run the pipeline
if __name__ == "__main__":
    events = fetch_all_events()
    save_events(events)

✅ 1 events pulled for venue 4112
✅ 0 events pulled for venue 1643
✅ 0 events pulled for venue 2064
✅ 0 events pulled for venue 4138
✅ 1 events pulled for venue 3012
✅ 0 events pulled for venue 262250
✅ 1 events pulled for venue 4112
✅ 0 events pulled for venue 1643
✅ 0 events pulled for venue 2064
✅ 0 events pulled for venue 4138
✅ 1 events pulled for venue 3012
✅ 0 events pulled for venue 262250
✅ Saved 4 events to events_data/KC_events/Raw\events_kc_20250730.csv
